In [1]:
import pandas as pd
import numpy as np
import pickle
from pandas_profiling import ProfileReport

In [ ]:
import json

In [ ]:
# transcription_df = pd.read_json('../data/public_train/train_transcriptions.json')
with open('../data/public_train/test_transcriptions.json') as file:
    train_transcripts = json.load(file)

In [ ]:
len(train_transcripts)

In [ ]:
transcription_df = pd.DataFrame(train_transcripts)

In [ ]:
transcription_df.head()

In [ ]:
transcription_df.img_id.values

In [ ]:
transcription_df.narration.astype(str).value_counts()

In [ ]:
transcription_df.dialog.astype(str).value_counts()

In [ ]:
transcription_df['text'] = transcription_df.apply(lambda x: x['dialog']+x['narration'],axis=1)

In [ ]:
transcription_df['text'].values

In [ ]:
len(transcription_df)

In [ ]:
transcription_df['text'] = transcription_df['text'].apply(lambda x: [y for y in x if type(y) == str])

In [ ]:
len(transcription_df)

In [ ]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"!", " !", phrase)
    phrase = re.sub(r"\.", " .", phrase)
    phrase = re.sub(r",", " ,", phrase)
    return phrase

In [ ]:
from fastpunct import FastPunct

In [ ]:
fastpunct = FastPunct()

In [ ]:
# transcription_df['text'][:10].apply(lambda x: fastpunct.punct(x,correct=True)).values

In [ ]:
# transcription_df = transcription_df[transcription_df['text'].astype(str)!='[]']

In [ ]:
transcription_df['text_clean'] = transcription_df['text'].apply(lambda x: fastpunct.punct(x,correct=True) if str(x) != '[]' else '[]')

In [ ]:
transcription_df[['text','text_clean']]

In [ ]:
transcription_df['text_clean'] = transcription_df['text_clean'].apply(lambda x: [decontracted(y).lower() for y in x])

In [ ]:
len(transcription_df)

In [ ]:
transcription_df.columns = ['dialog', 'narration', 'image_id', 'text','text_clean']

In [ ]:
transcription_df.head()

In [ ]:
len(transcription_df)

In [ ]:
# df = pd.merge(label_df,transcription_df,on='image_id',how='inner')

In [ ]:
# df.head()

In [ ]:
emotions = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral','Others']

In [ ]:
# df['emotion_list'] = df.apply(lambda x: [emo for emo in emotions if x[emo] == '1'],axis=1)

In [ ]:
transcription_df.to_csv('../data/public_train/public_test_dataset.csv',index=None)

In [2]:
transcription_df = pd.read_csv('../data/public_train/public_test_dataset.csv')

In [3]:
import cv2

def resize_img(path):
  try:
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32)/255
    return img
  except Exception as e:
    print(str(e))
    return None

In [4]:
base_path = '../data/public_train/test/'

In [5]:
test_imgs = []
i = 0
for index, row in transcription_df.iterrows():
  if i%100 == 0:
    print("Processing i:", i, base_path+str(row['image_id'])+'.jpg')
  img = resize_img(base_path+str(row['image_id'])+'.jpg')
  if img is not None:
    test_imgs.append(img)
  else:
#     test = test.drop(index)
    print("Dropping row:", index, "Length:", len(test))
  i += 1

Processing i: 0 ../data/public_train/test/1000_4_6.jpg
Processing i: 100 ../data/public_train/test/117_5_2.jpg
Processing i: 200 ../data/public_train/test/1327_37_4.jpg
Processing i: 300 ../data/public_train/test/1464_20_4.jpg
Processing i: 400 ../data/public_train/test/1661_16_4.jpg
Processing i: 500 ../data/public_train/test/1846_14_1.jpg
Processing i: 600 ../data/public_train/test/2032_16_2.jpg
Processing i: 700 ../data/public_train/test/2186_33_0.jpg
Processing i: 800 ../data/public_train/test/288_11_1.jpg
Processing i: 900 ../data/public_train/test/3833_14_5.jpg
Processing i: 1000 ../data/public_train/test/3777_5_3.jpg
Processing i: 1100 ../data/public_train/test/704_8_2.jpg
Processing i: 1200 ../data/public_train/test/2693_38_5.jpg
Processing i: 1300 ../data/public_train/test/888_45_3.jpg
Processing i: 1400 ../data/public_train/test/3080_4_1.jpg
Processing i: 1500 ../data/public_train/test/2403_47_7.jpg
Processing i: 1600 ../data/public_train/test/3493_8_5.jpg
Processing i: 1700 

In [6]:
assert len(transcription_df) == len(test_imgs)

In [7]:
test_np_imgs = np.array(test_imgs)
test_np_imgs.shape

(2046, 224, 224, 3)

In [8]:
with open('../data/public_train/public_test_np_img_norm','wb') as f: pickle.dump(test_np_imgs, f)